In [ ]:
# Generate a nicely formatted transactions/money table
from datetime import date
from re import split
import pandas as pd

from chtl_helpers import balance_to_float

def current_year():
    """Returns the current UTC year."""
    return date.today().year

def read_transactions_csv(filename):
    """Call pd.read_csv with appropriate options and file prefix."""
    money_cols = (
        'Payment Amount', 'Discount', 'On Account', 'Transaction Amount', 'Amount Due', 'Amount Paid',
         'Amount Due For Line Item', 'Actual Paid',
    )
    return pd.read_csv(
        f'data/input/{filename}',
        # Excludes colums redundant with another table.
        usecols=(
            'Transaction ID', 'Member ID', 'Date', 'Payment method', 'Payment Amount', 'Discount', 'On Account',
            # "Item Type" is actually the "Transaction Type" - confusing.
            'Transaction Amount', 'Amount Due', 'Amount Paid', 'Item Type', 'Amount Due For Line Item', 'Actual Paid',
            'Item ID', 'Membership Type', 'Handled By', 'Notes', 'Renewal',
        ),
        dtype={
            # Typically a number, but members can choose their own, e.g. "t00ln00b"
            'Member ID': 'string',
            # I manually modified two loans in loans-2017 that had "Item ID": "Drill" to use ID 517, rest should be Int64.
            # Without an explicit type, some files parse the "Item ID" as a number, some as a string.
            'Item ID': 'Int64',
            # For some reason this pops up as an object
            'Item Type': 'string',
        },
        # Convert "$0.00" strings to actual numbers
        converters={ col: balance_to_float for col in money_cols},
        # 'Checked Out'. 'Checked In', 'Due Date'
        parse_dates=['Date'],
    )

raw_transactions = pd.concat([
    read_transactions_csv(f'transactions-{year}.csv')
    for year in range(2016, current_year() + 1)
]).reset_index()

raw_transactions

In [ ]:
# "Item Type" is only set for the first row in a sequence of transactions.
# Use the initial value for each row for forward fill until the next value.
raw_transactions['Item Type'].fillna(method='ffill', inplace=True)
raw_transactions

In [ ]:
raw_transactions.to_csv('data/output/transactions.csv')
raw_transactions.to_pickle('data/output/transactions.pkl')